In [1]:
import time

In [2]:
from tensorflow.keras.models import load_model

2024-04-09 02:27:00.893957: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 02:27:01.447762: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from deap import base, creator, tools, algorithms

In [4]:
import numpy as np

In [5]:
from tqdm import tqdm

In [6]:
import tensorflow as tf

In [7]:
# import multiprocessing

In [7]:
loaded_model = load_model('surogate.h5')

2024-04-09 02:27:09.277030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-09 02:27:09.277533: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [8]:
# 特定频率
def loss_freq(s_para, disp=False):
    # 能量
    E = np.square(s_para)
    E11 = E[:,0] + E[:,1]
    E21 = E[:,2] + E[:,3]
    E31 = E[:,4] + E[:,5]
    E41 = E[:,6] + E[:,7]
    P21 = np.arctan2(s_para[:,3], s_para[:,2])
    P31 = np.arctan2(s_para[:,5], s_para[:,4])

    # 压制
    loss1 = E11 - E21 - E31 + E41
    # 比例
    loss2 = np.abs(E21 / (E31 + E21) - 0.7)
    # phase
    loss3 = np.abs(P21 - P31 - np.pi / 4)

    loss4 = np.abs(np.sum(E, axis = -1) - 1)

    if disp:
        print(f"P21:{P21}\nP31:{P31}")
        print(f"E11:{E11}\nE21:{E21}\nE31:{E31}\nE41:{E41}")
        print(f"loss1:{loss1}\nloss2:{loss2}\nloss3:{loss3}")
        return E11, E21, E31, E41, P21, P31, loss1, loss2, loss3
    
    return loss1 + loss2 + loss3 + loss4

In [9]:
def dgn_obj(s1, s2, s3):
    # max_loss = np.max([loss_freq(s1), loss_freq(s2), loss_freq(s3)], axis=0)
    # max_loss_tuple = [(value,) for value in max_loss]
    # return max_loss_tuple
    sum_loss = loss_freq(s1) + loss_freq(s2) + loss_freq(s3)
    sum_loss_tuple = [(value,) for value in sum_loss]
    return sum_loss_tuple

In [10]:
# 参数数量，迭代次数
NUM_NODES = 200
EPOCHS = 3000

In [11]:
# 定义个体表示方式
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)

In [12]:
toolbox = base.Toolbox()

In [13]:
# 物理边界
lb = np.array([1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4])
ub = np.array([5, 5, 5, 5, 5, 5, 5, 5, 100, 100, 100, 100])

In [14]:
f1 = np.ones((NUM_NODES, 1)) * 2.4
f2 = np.ones((NUM_NODES, 1)) * 2.5
f3 = np.ones((NUM_NODES, 1)) * 2.6

In [15]:
toolbox.register("attr_float", np.random.uniform, low=lb, high=ub)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [16]:
def evaluate(x):
    geoms = np.array(x)
    n_samples = geoms.shape[0]
    inputs1 = np.concatenate((f1[:n_samples], geoms), axis=-1)
    inputs2 = np.concatenate((f2[:n_samples], geoms), axis=-1)
    inputs3 = np.concatenate((f3[:n_samples], geoms), axis=-1)
    y_pred1 = loaded_model.predict(inputs1, batch_size=n_samples, verbose=0)
    y_pred2 = loaded_model.predict(inputs2, batch_size=n_samples, verbose=0)
    y_pred3 = loaded_model.predict(inputs3, batch_size=n_samples, verbose=0)
    return dgn_obj(y_pred1, y_pred2, y_pred3)

In [17]:
toolbox.register("evaluate", evaluate)

In [19]:
# def init_worker(worker_id):
#     global loaded_model
#     loaded_model = load_model('surogate.h5')

In [20]:
# pool = multiprocessing.Pool(initializer=init_worker, initargs=(0,), processes=multiprocessing.cpu_count())
# toolbox.register("map", pool.map)

In [18]:
def checkBounds():
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                mask = np.where((child < lb) | (child > ub))
                child[mask] = np.random.uniform(lb[mask], ub[mask])
            return offspring
        return wrapper
    return decorator

In [19]:
# 注册选择、交叉和突变操作
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [20]:
toolbox.decorate("mate", checkBounds())
toolbox.decorate("mutate", checkBounds())

In [21]:
population = toolbox.population(n=NUM_NODES)
halloffame = tools.HallOfFame(1, similar=np.array_equal)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

In [22]:
cxpb = 0.5
mutpb = 0.2

In [23]:
logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

In [24]:
# Evaluate the individuals with an invalid fitness
invalid_ind = [ind for ind in population if not ind.fitness.valid]
fitnesses = toolbox.evaluate(invalid_ind)
for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit

In [25]:
halloffame.update(population)

In [26]:
record = stats.compile(population)

In [27]:
logbook.record(gen=0, nevals=len(invalid_ind), **record)

In [28]:
print(logbook.stream)
# Begin the generational process
logs = []
t0 = time.time()
for gen in tqdm(range(1, EPOCHS + 1)):
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Vary the pool of individuals
    offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.evaluate(invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Update the hall of fame with the generated individuals
    halloffame.update(offspring)

    # Replace the current population by the offspring
    population[:] = offspring

    # Append the current generation statistics to the logbook
    record = stats.compile(population)
    logbook.record(gen=gen, nevals=len(invalid_ind), **record)

    logs.append({'epoch': gen, 'time': time.time() - t0, 'acc': halloffame[0].fitness})
    # print(logbook.stream)

gen	nevals	avg    	min     	max    
0  	200   	7.45935	-0.02187	18.8869


100%|███████████████████████████████████████| 3000/3000 [05:32<00:00,  9.02it/s]


In [29]:
logs

[{'epoch': 1,
  'time': 0.15385842323303223,
  'acc': deap.creator.FitnessMin((-1.4849750995635986,))},
 {'epoch': 2,
  'time': 0.26887035369873047,
  'acc': deap.creator.FitnessMin((-1.4849750995635986,))},
 {'epoch': 3,
  'time': 0.3986189365386963,
  'acc': deap.creator.FitnessMin((-1.4849750995635986,))},
 {'epoch': 4,
  'time': 0.5145771503448486,
  'acc': deap.creator.FitnessMin((-1.4849750995635986,))},
 {'epoch': 5,
  'time': 0.6352112293243408,
  'acc': deap.creator.FitnessMin((-1.4849750995635986,))},
 {'epoch': 6,
  'time': 0.753758430480957,
  'acc': deap.creator.FitnessMin((-1.632054328918457,))},
 {'epoch': 7,
  'time': 0.8670430183410645,
  'acc': deap.creator.FitnessMin((-1.632054328918457,))},
 {'epoch': 8,
  'time': 0.9761185646057129,
  'acc': deap.creator.FitnessMin((-1.632054328918457,))},
 {'epoch': 9,
  'time': 1.0937633514404297,
  'acc': deap.creator.FitnessMin((-2.0607125759124756,))},
 {'epoch': 10,
  'time': 1.2004261016845703,
  'acc': deap.creator.FitnessM

In [33]:
for log in logs:
    print(log['time'], log['acc'])

0.15385842323303223 (-1.4849750995635986,)
0.26887035369873047 (-1.4849750995635986,)
0.3986189365386963 (-1.4849750995635986,)
0.5145771503448486 (-1.4849750995635986,)
0.6352112293243408 (-1.4849750995635986,)
0.753758430480957 (-1.632054328918457,)
0.8670430183410645 (-1.632054328918457,)
0.9761185646057129 (-1.632054328918457,)
1.0937633514404297 (-2.0607125759124756,)
1.2004261016845703 (-2.0607125759124756,)
1.3072476387023926 (-2.163675546646118,)
1.4183502197265625 (-2.163675546646118,)
1.5307331085205078 (-2.222195863723755,)
1.63627290725708 (-2.2822489738464355,)
1.7552733421325684 (-2.2822489738464355,)
1.8736093044281006 (-2.4640610218048096,)
1.977890968322754 (-2.4640610218048096,)
2.082122564315796 (-2.4640610218048096,)
2.1919639110565186 (-2.4640610218048096,)
2.29811954498291 (-2.4640610218048096,)
2.4125616550445557 (-2.4640610218048096,)
2.5213146209716797 (-2.4640610218048096,)
2.6471996307373047 (-2.4640610218048096,)
2.7598073482513428 (-2.4640610218048096,)
2.8

In [30]:
print("Best individual is: %s\nwith fitness: %s" % (halloffame[0], halloffame[0].fitness))

Best individual is: [ 2.06787199  1.44907122  1.51895104  2.35452722  3.55813324  1.01322259
  2.44557659  3.60756171 18.3983113  36.06556786  8.21286129 89.89663154]
with fitness: (-2.647954225540161,)


In [ ]:
# import matplotlib.pyplot as plt
# gen, avg, min_, max_ = logbook.select("gen", "avg", "min", "max")
# plt.plot(gen, avg, label="average")
# plt.plot(gen, min_, label="minimum")
# plt.plot(gen, max_, label="maximum")
# plt.xlabel("Generation")
# plt.ylabel("Fitness")
# plt.legend(loc="lower right")
# plt.show()

In [ ]:
geom = np.array([halloffame[0]])
input = np.concatenate([np.array([[2.4], [2.5], [2.6]]), np.repeat(geom, 3, axis=0)], axis=-1)

In [ ]:
best_para = loaded_model.predict(input, batch_size=3, verbose=0)

In [ ]:
best_para

In [ ]:
actu_para = np.array([[-0.144, 0.019, 0.667, 0.393, 0.543, -0.157, 0.152, -0.12],
                      [-0.191, -0.036, 0.791, 0.16, 0.435, -0.292, 0.003, -0.115],
                      [-0.245, -0.084, 0.809, -0.12, 0.312, -0.368, -0.09, -0.012]])

In [ ]:
mse(best_para, actu_para)

In [ ]:
np.mean((best_para - actu_para)[1])

In [ ]:
best_para - actu_para

In [ ]:
actuLoss = loss_freq(actu_para, disp=True)

In [ ]:
0.5 * actuLoss[-3] + 0.03 * actuLoss[-2] + 0.47 * actuLoss[-1]

In [ ]:
X_test = np.array([[2.4, 2.27248, 2.52657, 1.06926, 3.83355, 1.92307, 1.74249, 1.37051,  1.52846,  17.2486, 66.9054, 18.5276, 50.3882],
                   [2.5, 2.27248, 2.52657, 1.06926, 3.83355, 1.92307, 1.74249, 1.37051,  1.52846,  17.2486, 66.9054, 18.5276, 50.3882],
                   [2.6, 2.27248, 2.52657, 1.06926, 3.83355, 1.92307, 1.74249, 1.37051,  1.52846,  17.2486, 66.9054, 18.5276, 50.3882]])
y_test = np.array([[0.016, 0.075, 0.743, 0.343, 0.496, -0.185, -0.147, 0.005],
                   [-0.021, 0.003, 0.827, 0.089, 0.414, -0.333, -0.026, -0.018],
                   [-0.093, -0.046, 0.802, -0.177, 0.281, -0.444, 0.052, -0.109]])

In [ ]:
s_para = loaded_model.predict(X_test, batch_size=3, verbose=0)

In [ ]:
s_para - y_test

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()

In [ ]:
class EnergyMSELoss2(tf.keras.losses.Loss):
    def __init__(self, name='energy_mse_loss2'):
        super().__init__(name=name)
        self.mse = tf.keras.losses.MeanSquaredError()

    def call(self, y_true, y_pred):
        E_y_true = tf.reduce_sum(tf.square(y_true), axis=1)
        E_y_pred = tf.reduce_sum(tf.square(y_pred), axis=1)
        myMse = self.mse(y_true, y_pred)
        EnergyLoss = tf.reduce_mean(tf.square(E_y_true - E_y_pred))
        print(myMse)
        print(EnergyLoss)
        return myMse + 0.1 * EnergyLoss

    def get_config(self):
        base_config = super().get_config()
        return base_config

In [ ]:
myLoss = EnergyMSELoss2()

In [ ]:
y_test = y_test.astype(np.float32)

In [ ]:
myLoss(y_test, s_para)

In [ ]:
testLoss = loss_freq(s_para, disp=True)

In [ ]:
0.5 * testLoss[-3] + 0.03 * testLoss[-2] + 0.47 * testLoss[-1]

In [ ]:
np.max([0.038788, 0.0136575, 0.03736])

In [ ]:
np.max([0.007129329958, 0.0145997658, 0.013663119645])

In [ ]:
np.max([0.00057959177, 0.000114086254, 0.000556561867])

In [ ]:
num_nodes = 1000000

In [ ]:
# 物理边界
mmin = np.array([1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4])
mmax = np.array([5, 5, 5, 5, 5, 5, 5, 5, 100, 100, 100, 100])

In [ ]:
geom = np.random.uniform(mmin, mmax, (num_nodes, 12))
freq1 = np.ones((num_nodes, 1), dtype=np.float32) * 2.4
freq2 = np.ones((num_nodes, 1), dtype=np.float32) * 2.5
freq3 = np.ones((num_nodes, 1), dtype=np.float32) * 2.6

In [ ]:
y_pred1 = loaded_model.predict(np.concatenate((freq1, geom), axis=-1), batch_size=num_nodes, verbose=0)
y_pred2 = loaded_model.predict(np.concatenate((freq2, geom), axis=-1), batch_size=num_nodes, verbose=0)
y_pred3 = loaded_model.predict(np.concatenate((freq3, geom), axis=-1), batch_size=num_nodes, verbose=0)

In [ ]:
D = loss_freq(y_pred2, disp=True)

In [ ]:
L1, L2, L3 = D[-3], D[-2], D[-1]

In [ ]:
np.max(D[0] + D[1] + D[2] + D[3])

In [ ]:
plt.hist(D[0] + D[1] + D[2] + D[3], bins=100)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(0.5*L1+0.03*L2+0.47*L3, bins=100)

In [ ]:
plt.hist(L2, bins=100)

In [ ]:
import pandas as pd

In [ ]:
AS_dataset = pd.read_csv('./../Arbitrary_Single_band_Coupler_Phase_Shift.csv', encoding='utf-8')
full_X = AS_dataset.loc[:,'freq':'L4'].to_numpy(dtype = np.float32)
full_y = AS_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)

In [ ]:
pred_y = loaded_model.predict(full_X, batch_size=4096, verbose=0)

In [ ]:
D = loss_freq(pred_y, disp=True)

In [ ]:
plt.hist(D[0] + D[1] + D[2] + D[3], bins=100)